In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('vacancies.xlsx')

In [3]:
df = pd.DataFrame(data)
print(df.columns)

Index(['Unnamed: 0', 'requisitionId', 'internalId', 'title', 'company',
       'companyShortName', 'publicationDate', 'introduction', 'specialization',
       'specializationId', 'duties', 'requirements', 'conditions',
       'acceptHandicappedFlag', 'city', 'region', 'salary_min', 'salary_max',
       'workScheduleId', 'experienceId', 'link'],
      dtype='object')


In [4]:
df = df.astype(str)

In [5]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=1878a3ec83cdd12937cccdd4d3928df863c249a475b1872ba6cb39f5d15f4723
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [6]:
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import re

In [7]:

# Загрузка ресурсов NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Инициализация морфологического анализатора pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Функция для предобработки текста
def preprocess_text(text):
    # Токенизация текста с использованием токенизатора punkt
    tokens = word_tokenize(text.lower(), language='russian')

    # Объявление стоп слов
    stop_words = set(stopwords.words('russian'))
    # Удаление знаков препинания как отдельных токенов и стоп слов
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    tokens= [token for token in tokens if re.match(r'^[\w\d]+$', token)]


    # Лемматизация токенов
    lemmatized_tokens = [morph.parse(word)[0].normal_form for word in tokens]

    return lemmatized_tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# Предобработка текста в переменных specialization, duties, requirements
preprocessed_specialization = df['specialization'].apply(preprocess_text)
preprocessed_duties = df['duties'].apply(preprocess_text)
preprocessed_requirements = df['requirements'].apply(preprocess_text)
preprocessed_title = df['title'].apply(preprocess_text)
preprocessed_introduction = df['introduction'].apply(preprocess_text)


# Вывод результата предобработки
print("Preprocessed Specialization:")
print(preprocessed_specialization)
print("\nPreprocessed Duties:")
print(preprocessed_duties)
print("\nPreprocessed Requirements:")
print(preprocessed_requirements)
print("Preprocessed_title:")
print(preprocessed_title)
print("Preprocessed_introduction:")
print(preprocessed_introduction)

Preprocessed Specialization:
0                                 [it, разработка]
1                                         [другой]
2                           [управление, персонал]
3                                         [другой]
4                                 [it, разработка]
                           ...                    
7495                        [it, тестирование, qa]
7496                        [it, тестирование, qa]
7497    [it, поддержка, devops, администрирование]
7498                        [бизнес, обслуживание]
7499                    [внутренний, безопасность]
Name: specialization, Length: 7500, dtype: object

Preprocessed Duties:
0       [предстоять, заниматься, работать, больший, об...
1       [выполнять, служебный, поручение, непосредстве...
2       [анализ, эффективность, применение, действоват...
3       [целеполагание, сбор, консолидация, приоритиза...
4       [разработка, прикладной, программа, java, поис...
                              ...               

In [29]:
resume='Ответственность, техническое обслуживание автомобилей, деловой этикет, стаж вождения от 5 лет, хорошее знание города, безаварийное вождение, правила дорожного движения, перевозка грузов, перевозка пассажиров, работа с сопроводительной документацией, спокойный стиль вождения, выполнение поручений руководителя, содержание автомобиля в чистоте, работа на автомобилях представительского класса'
preprocessed_resume= preprocess_text(resume)
print(preprocessed_resume)

['ответственность', 'технический', 'обслуживание', 'автомобиль', 'деловой', 'этикет', 'стаж', 'вождение', '5', 'год', 'хороший', 'знание', 'город', 'безаварийный', 'вождение', 'правило', 'дорожный', 'движение', 'перевозка', 'груз', 'перевозка', 'пассажир', 'работа', 'сопроводительный', 'документация', 'спокойный', 'стиль', 'вождение', 'выполнение', 'поручение', 'руководитель', 'содержание', 'автомобиль', 'чистота', 'работа', 'автомобиль', 'представительский', 'класс']


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from gensim.models import Word2Vec, FastText
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
# Объединяем списки текстов из всех столбцов в один список строк
combined_texts = df['preprocessed_introduction'].astype(str) + ' ' + \
                df['preprocessed_title'].astype(str) + ' ' + \
                df['preprocessed_requirements'].astype(str) + ' ' + \
                df['preprocessed_duties'].astype(str) + ' ' + \
                df['preprocessed_specialization'].astype(str)



In [13]:
combined_texts.to_excel('combined_texts.xlsx')

In [14]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, FastText
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# TF-IDF векторизация
vectorizer = TfidfVectorizer()
vectors_tfidf = vectorizer.fit_transform(combined_texts)
vector_tfidf_resume = vectorizer.transform([str(preprocessed_resume)])

# Word2Vec
text = ' '.join(preprocessed_resume)
sentences_resume = text.split()
sentences = combined_texts.apply(lambda x: x.split()).tolist()
model_word2vec = Word2Vec(sentences, min_count=1)
model_word2vec_resume = Word2Vec(sentences_resume, min_count=1)
vectors_word2vec = np.array([model_word2vec.wv[word] for sentence in sentences for word in sentence])
vectors_word2vec_resume= np.array([model_word2vec_resume.wv[word] for sentence in sentences_resume for word in sentence])

# FastText
model_fasttext = FastText(sentences, min_count=1)
model_fasttext_resume = FastText(sentences_resume, min_count=1)
vectors_fasttext = np.array([model_fasttext.wv[word] for sentence in sentences for word in sentence])
vectors_fasttext_resume = np.array([model_fasttext_resume.wv[word] for sentence in sentences_resume for word in sentence])


In [17]:
# Ваш вектор резюме (предположим, что он имеет размерность (1, n))
vector_resume = [vector_tfidf_resume,vectors_word2vec_resume,vectors_fasttext_resume]

# Ваши вектора описаний вакансий (предположим, что они хранятся в списке vectors_vacancies)
vectors_vacancies = [vectors_tfidf,vectors_word2vec,vectors_fasttext]
print("Размерность вектора резюме:")
for vec_resume in vector_resume:
    print(vec_resume.shape)
print("Размерность векторов описаний вакансий:")
for vec_vacancy in vectors_vacancies:
    print(vec_vacancy.shape)

Размерность вектора резюме:
(1, 13688)
(320, 100)
(320, 100)
Размерность векторов описаний вакансий:
(7500, 13688)
(816980, 100)
(816980, 100)


In [18]:
similarity_tfidf = cosine_similarity(vectors_tfidf, vector_tfidf_resume)
similarity_word2vec = cosine_similarity(vectors_word2vec, vectors_word2vec_resume)
similarity_fasttext = cosine_similarity(vectors_fasttext, vectors_fasttext_resume)

In [19]:
print('similarity_tfidf: ',similarity_tfidf)
print()
print('similarity_word2vec: ', similarity_word2vec)
print()
print('similarity_fasttext: ', similarity_fasttext)

similarity_tfidf:  [[0.00824548]
 [0.05142096]
 [0.003992  ]
 ...
 [0.02910642]
 [0.01267665]
 [0.0039139 ]]

similarity_word2vec:  [[ 0.2904368  -0.03750926  0.13431267 ...  0.04617491  0.21836887
   0.21836887]
 [ 0.12136912  0.11617251  0.10721442 ...  0.12429921  0.16890678
   0.16890678]
 [ 0.25002587  0.05381867  0.20739815 ...  0.09116582  0.12182702
   0.12182702]
 ...
 [ 0.07530848 -0.09481372  0.15857527 ...  0.05089846  0.05932298
   0.05932298]
 [ 0.08231365 -0.31388485  0.09266431 ...  0.03252022  0.04483069
   0.04483069]
 [ 0.0837983  -0.34308103  0.0237739  ...  0.11520845  0.02723429
   0.02723429]]

similarity_fasttext:  [[ 0.10553807 -0.05147865 -0.03388921 ...  0.09966929 -0.13314122
  -0.13314122]
 [-0.01888577 -0.04416866 -0.01529807 ... -0.08610129 -0.0651266
  -0.0651266 ]
 [-0.00738227  0.07192467 -0.14824146 ... -0.03848694 -0.05859661
  -0.05859661]
 ...
 [-0.13877891  0.18178526  0.07504131 ...  0.19018812 -0.09468149
  -0.09468149]
 [-0.04874064 -0.06914536

In [20]:
#def cosine_similarity_to_percentage(cosine_similarity):
    # Преобразование значения косинусной близости в проценты
    #similarity_percentage = (cosine_similarity + 1) * 50
    #return similarity_percentage

#def calculate_overall_similarity(similarity_values):
    # Вычисление среднего значения сходства в процентах
    #overall_similarity = np.mean(similarity_values)
    #return overall_similarity


In [21]:
# Преобразование значений косинусной близости в проценты
#similarity_percentages_tfidf = cosine_similarity_to_percentage(similarity_tfidf)
#similarity_percentages_word2vec = cosine_similarity_to_percentage(similarity_word2vec)
#similarity_percentages_tfasttext = cosine_similarity_to_percentage(similarity_fasttext)

# Вычисление общего процента сходства
#overall_similarity_percentage_tfidf = calculate_overall_similarity(similarity_percentages_tfidf)
#overall_similarity_percentage_word2vec = calculate_overall_similarity(similarity_percentages_word2vec)
#overall_similarity_percentage_fasttext = calculate_overall_similarity(similarity_percentages_tfasttext)

# Вывод результата
#print(f"Общий процент сходства tfidf: {overall_similarity_percentage_tfidf:.2f}%")
#print(f"Общий процент сходства word2vec: {overall_similarity_percentage_word2vec:.2f}%")
#print(f"Общий процент сходства fasttext: {overall_similarity_percentage_fasttext:.2f}%")

Общий процент сходства tfidf: 50.90%
Общий процент сходства word2vec: 52.11%
Общий процент сходства fasttext: 49.42%


In [22]:
#def calculate_similarity(vectors_vacancy, vector_resume):
     #Вычисляем косинусную близость между вектором вакансии и вектором резюме
    #similarity = cosine_similarity(vectors_vacancy, vector_resume)
    #return similarity

#def rank_vacancies(similarity_scores):
     #Ранжируем вакансии по мере их сходства с опытом
     #ranked_indices = similarity_scores.argsort(axis=0)[::-1].squeeze()
     #return ranked_indices

#def get_top_n_vacancies(ranked_indices, n=10):
    # Получаем Топ-N наиболее релевантных вакансий
    #top_n_vacancies = ranked_indices[:n]
    #return top_n_vacancies

In [23]:
 #Посчитем меру сходства для опыта и описаний вакансий для tfidf
#similarity_scores = calculate_similarity(vectors_tfidf, vector_tfidf_resume)

# Ранжировать вакансии по мере их сходства с опытом
#ranked_indices = rank_vacancies(similarity_scores)

# Вывести Топ-10 наиболее релевантных вакансий
#top_10_vacancies = get_top_n_vacancies(ranked_indices)
#print("Топ-10 наиболее релевантных вакансий:")
#for i, idx in enumerate(top_10_vacancies):
    #print(f"Вакансия {i+1}: {idx}, сходство: {similarity_scores[idx]}")

Топ-10 наиболее релевантных вакансий:
Вакансия 1: 6033, сходство: [0.34244155]
Вакансия 2: 2951, сходство: [0.33833966]
Вакансия 3: 950, сходство: [0.31062924]
Вакансия 4: 2538, сходство: [0.30488937]
Вакансия 5: 3311, сходство: [0.28879322]
Вакансия 6: 5300, сходство: [0.27772508]
Вакансия 7: 3046, сходство: [0.24716685]
Вакансия 8: 1631, сходство: [0.24716685]
Вакансия 9: 3462, сходство: [0.24518714]
Вакансия 10: 4123, сходство: [0.23041816]


In [24]:
#top_10_vacancies = ranked_indices[:10]  # Получаем Топ-10 индексов вакансий

# Получаем строки для всех 10 вакансий в Топ-10 и выбираем нужные столбцы
#top_10_vacancies_info = df.iloc[top_10_vacancies][['title', 'introduction', 'specialization', 'duties', 'requirements', 'conditions', 'link']]

# Выводим информацию о Топ-10 вакансиях
#print("Топ-10 наиболее релевантных вакансий:")
#for i, (index, row) in enumerate(top_10_vacancies_info.iterrows(), 1):
    #print(f"Вакансия {i}:")
    #print(row)  # Выводим информацию о вакансии
    #print()

Топ-10 наиболее релевантных вакансий:
Вакансия 1:
title                                              Офисный водитель
introduction      Хочешь работать в крупной компании, ценишь ста...
specialization                                               Другое
duties            *   заблаговременно планировать оптимальные ма...
requirements      *   образование от среднего профессионального\...
conditions        *   стабильный оклад и социальная поддержка со...
link                         https://rabota.sber.ru/search/4256358/
Name: 6033, dtype: object

Вакансия 2:
title                                                      Водитель
introduction                       В штат банка требуется водитель:
specialization                            Административный персонал
duties              \n\n*   обеспечивать комфортное и безопасное...
requirements      *   образование от среднего профессионального ...
conditions        *   адрес: Беговая, 209\n*   график работы: пн...
link                       

In [25]:

class SimilarityRanker:
    def __init__(self, vectors_vacancy, vector_resume, df):
        self.vectors_vacancy = vectors_vacancy
        self.vector_resume = vector_resume
        self.df = df

    def calculate_similarity(self):
        # Вычисляем косинусную близость между вектором вакансии и вектором резюме
        similarity = cosine_similarity(self.vectors_vacancy, self.vector_resume)
        return similarity

    def rank_vacancies(self, similarity_scores):
        # Ранжируем вакансии по мере их сходства с опытом
        ranked_indices = similarity_scores.argsort(axis=0)[::-1].squeeze()
        return ranked_indices

    def get_top_n_vacancies(self, ranked_indices, n=10):
        # Получаем Топ-N наиболее релевантных вакансий
        top_n_vacancies = ranked_indices[:n]
        return top_n_vacancies

    def find_top_n_relevant_vacancies(self, n=10):
        # Посчитываем меру сходства для опыта и описаний вакансий
        similarity_scores = self.calculate_similarity()

        # Ранжировать вакансии по мере их сходства с опытом
        ranked_indices = self.rank_vacancies(similarity_scores)

        # Получить Топ-N наиболее релевантных вакансий
        top_n_vacancies = self.get_top_n_vacancies(ranked_indices, n)

        # Получить строки для всех 10 вакансий в Топ-10 и выбрать нужные столбцы
        top_n_vacancies_info = self.df.iloc[top_n_vacancies][['title', 'introduction', 'specialization', 'duties', 'requirements', 'conditions', 'link']]

        return top_n_vacancies_info

In [26]:

# Создаем объект класса
similarity_ranker = SimilarityRanker(vectors_tfidf, vector_tfidf_resume, df)

# Получаем Топ-10 наиболее релевантных вакансий
top_10_vacancies_info = similarity_ranker.find_top_n_relevant_vacancies()

# Выводим информацию о Топ-10 вакансиях
print("Топ-10 наиболее релевантных вакансий:")
print(top_10_vacancies_info)

Топ-10 наиболее релевантных вакансий:
                                                  title  \
6033                                   Офисный водитель   
2951                                           Водитель   
950                               Персональный водитель   
2538                                           Водитель   
3311                            Водитель (персональный)   
5300                            Водитель (персональный)   
3046                          Водитель мобильного офиса   
1631                          Водитель мобильного офиса   
3462  Старший водитель автомобиля, предназначенного ...   
4123                            Водитель (персональный)   

                                           introduction  \
6033  Хочешь работать в крупной компании, ценишь ста...   
2951                   В штат банка требуется водитель:   
950                                                  \*   
2538  Команда транспортного обеспечения Тульского от...   
3311  Сбер — лучш

In [30]:
resume=str(input())
preprocessed_resume= preprocess_text(resume)


Погрузочно-разгрузочные работы, права на погрузчик, работа на погрузчике, управление автопогрузчиком, работа на дизельном погрузчике, работа на вилочном погрузчике, права на управление погрузчиком, водитель погрузчика 


In [31]:
vector_tfidf_resume = vectorizer.transform([str(preprocessed_resume)])
text = ' '.join(preprocessed_resume)
sentences_resume = text.split()
vectors_word2vec_resume= np.array([model_word2vec_resume.wv[word] for sentence in sentences_resume for word in sentence])
model_fasttext_resume = FastText(sentences_resume, min_count=1)
vectors_fasttext_resume = np.array([model_fasttext_resume.wv[word] for sentence in sentences_resume for word in sentence])


In [33]:
from tabulate import tabulate

In [ ]:

# Создаем объект класса
similarity_ranker = SimilarityRanker(vectors_fasttext, vectors_fasttext, df)

# Получаем Топ-10 наиболее релевантных вакансий
top_10_vacancies_info = similarity_ranker.find_top_n_relevant_vacancies()

# Выводим информацию о Топ-10 вакансиях
print("Топ-10 наиболее релевантных вакансий:")
print(top_10_vacancies_info)

In [ ]:
table_str = tabulate(top_10_vacancies_info, headers='keys', tablefmt='grid')

# Вывод таблицы с границами
print(table_str)